In [1]:
! pip install plotly
! pip install umap-learn

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/0b/f8/b65cdd2be32e442c4efe7b672f73c90b05eab5a7f3f4115efe181d432c60/plotly-5.22.0-py3-none-any.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/e3/ee/b179c3ab5cb842d75c65339c4b86b572eaf8f43407890bd1d2c7b72eb829/tenacity-8.4.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB 435.7 kB/s eta 0:00:38
   ---------------------------------------- 0.1/16.4 MB 656.4 kB/s eta 0:00:25
   ---------------------------------------- 0.1/16.4 MB 901.1 kB/s eta 0:00:19
   ---------------------------------------- 0.2/16.4 MB 1.1 MB/s eta 0:00:16
    --------------------------------------- 0.3/16.4 MB 1.2 MB/s eta 0:00:14
    --------------------------------------- 0.3/16.4 MB 1.2 MB/


[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for umap-learn from https://files.pythonhosted.org/packages/d1/1b/46802a050b1c55d10c4f59fc6afd2b45ac9b4f62b2e12092d3f599286f14/umap_learn-0.5.6-py3-none-any.whl.metadata
  Obtaining dependency information for numba>=0.51.2 from https://files.pythonhosted.org/packages/79/89/2d924ca60dbf949f18a6fec223a2445f5f428d9a5f97a6b29c2122319015/numba-0.60.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pynndescent>=0.5 from https://files.pythonhosted.org/packages/d2/53/d23a97e0a2c690d40b165d1062e2c4ccc796be458a1ce59f6ba030434663/pynndescent-0.5.13-py3-none-any.whl.metadata
  Obtaining dependency information for llvmlite<0.44,>=0.43.0dev0 from https://files.pythonhosted.org/packages/20/ab/ed5ed3688c6ba4f0b8d789da19fd8e30a9cf7fc5852effe311bc5aefe73e/llvmlite-0.43.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/85.7 kB ? eta -:--:--
   ------------------- -------------------- 41.0/85.7 kB 960.0 kB/s et


[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import os
from sklearn.manifold import TSNE
import time

In [31]:
def read_embeddings(artifact_uri, dataset, epoch):
    
    print(os.path.join(artifact_uri, f'{dataset}_enrolled_embeddings.npz'))
    enrolled_embeddings = np.load(os.path.join(artifact_uri, f'{dataset}_enrolled_embeddings_{epoch}.npz'))['arr_0']
    enrolled_labels = np.load(os.path.join(artifact_uri, f'{dataset}_enrolled_labels_{epoch}.npz'))['arr_0']
    enrolled_scan_ids = np.load(os.path.join(artifact_uri, f'{dataset}_enrolled_scan_ids_{epoch}.npz'))['arr_0']
    enrolled_perspectives = np.load(os.path.join(artifact_uri, f'{dataset}_enrolled_perspectives_{epoch}.npz'))['arr_0']
    query_embeddings = np.load(os.path.join(artifact_uri, f'{dataset}_query_embeddings_{epoch}.npz'))['arr_0']
    query_labels = np.load(os.path.join(artifact_uri, f'{dataset}_query_labels_{epoch}.npz'))['arr_0']
    query_scan_ids = np.load(os.path.join(artifact_uri, f'{dataset}_query_scan_ids_{epoch}.npz'))['arr_0']
    query_perspectives = np.load(os.path.join(artifact_uri, f'{dataset}_query_perspectives_{epoch}.npz'))['arr_0']
    distances = None #np.load(os.path.join(artifact_uri, f'{dataset}_distances_{epoch}.npz'))
    
    
    return (enrolled_embeddings, enrolled_labels, enrolled_scan_ids, enrolled_perspectives, 
            query_embeddings, query_labels, query_scan_ids, query_perspectives, distances)

In [33]:
#artifact_uri = "F:\\Face\\HM_IDENT_3DFR\\src\\log\\mlruns\\633479264740573887\\7850d734dc134028ba19e52e3f37a601\\artifacts\\embeddings"
artifact_uri = "..\\..\\log\\mlruns\\113129204246726823\\9af1ded0cf884408998dec982035f5c9\\artifacts\\embeddings\\"
dataset = 'bellus'
epoch = 40
enrolled_embeddings, enrolled_labels, enrolled_scan_ids, enrolled_perspectives, query_embeddings, query_labels, query_scan_ids, query_perspectives, distances = read_embeddings(artifact_uri, dataset, epoch)
print(enrolled_embeddings)

..\..\log\mlruns\113129204246726823\9af1ded0cf884408998dec982035f5c9\artifacts\embeddings\bellus_enrolled_embeddings.npz
[[ 0.33282456  1.0828675   0.6561941  ...  0.21605776  1.123315
   0.10464958]
 [ 1.4531662  -1.1061554   0.83027023 ... -0.56982756 -1.2881137
  -0.1950432 ]
 [-1.0700661   0.97439706 -0.88170207 ...  0.6283077   0.9527299
   0.48865476]
 ...
 [-1.1150627   0.463378    0.85441566 ... -0.04176355  0.35211188
  -1.4405446 ]
 [ 0.6183775   1.220677   -0.1401561  ...  1.0728538  -1.1654183
   1.7575132 ]
 [ 0.521437   -1.1236801  -1.0086526  ... -0.10932133  1.6305763
   0.48261544]]


In [34]:

start = time.time()
tsne = TSNE(n_components = 3, random_state=0)
projections = tsne.fit_transform(enrolled_embeddings)
end = time.time()
print(f"generating projections with T-SNE took: {(end-start):.2f} sec")

generating projections with T-SNE took: 14.23 sec


In [35]:
import plotly.graph_objects as go
import plotly.express as px

from typing import Dict
from pathlib import Path
from IPython.core.display import display, HTML

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)


def display_projections(
    labels: np.ndarray,
    projections: np.ndarray,
    #image_paths: np.ndarray,
    #image_data_uris: Dict[str, str],
    show_legend: bool = False,
    show_markers_with_text: bool = True
) -> None:
    # Create a separate trace for each unique label
    unique_labels = np.unique(labels)
    traces = []
    for unique_label in unique_labels:
        mask = labels == unique_label
        #customdata_masked = image_paths[mask]
        trace = go.Scatter3d(
            x=projections[mask][:, 0],
            y=projections[mask][:, 1],
            z=projections[mask][:, 2],
            mode='markers+text' if show_markers_with_text else 'markers',
            text=labels[mask],
            #customdata=customdata_masked,
            name=str(unique_label),
            marker=dict(size=8),
            hovertemplate="<b>class: %{text}</b><br>path: %{customdata}<extra></extra>"
        )
        traces.append(trace)

    # Create the 3D scatter plot
    fig = go.Figure(data=traces)
    fig.update_layout(
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
        width=1000,
        height=1000,
        showlegend=show_legend
    )

    # Convert the chart to an HTML div string and add an ID to the div
    plotly_div = fig.to_html(full_html=False, include_plotlyjs=False, div_id="scatter-plot-3d")


    # Create an HTML template including the chart div and JavaScript code
    html_template = f"""
    <!DOCTYPE html>
    <html>
        <head>
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                #image-container {{
                    position: fixed;
                    top: 0;
                    left: 0;
                    width: 200px;
                    height: 200px;
                    padding: 5px;
                    border: 1px solid #ccc;
                    background-color: white;
                    z-index: 1000;
                    box-sizing: border-box;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    text-align: center;
                }}
                #image-display {{
                    width: 100%;
                    height: 100%;
                    object-fit: contain;
                }}
            </style>
        </head>        
        <body>
            {plotly_div}
        </body>
    </html>
    """

    # Display the HTML template in the Jupyter Notebook
    display(HTML(html_template))

C:\Users\Eduard\AppData\Local\Temp\ipykernel_8184\1857090004.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [36]:
display_projections(
    labels=enrolled_labels,
    projections=projections
    #image_paths=image_paths,
    #image_data_uris=image_data_uris
)

In [ ]:
import umap

start = time.time()
projections = umap.UMAP(n_components=3).fit_transform(enrolled_embeddings)
end = time.time()
print(f"generating projections with UMAP took: {(end-start):.2f} sec")

display_projections(
    labels=enrolled_labels,
    projections=projections
)

In [ ]:
def normalize(vec: np.ndarray):
    return vec / np.linalg.norm(vec, axis=1, keepdims=True)

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from sklearn.preprocessing import normalize

# Sample Data (for illustration, replace with your actual data)
# enrolled_embeddings = np.random.rand(10, 128)  # Replace with your actual embeddings
# enrolled_labels = ['Label1', 'Label2', 'Label1', 'Label3', 'Label2', 'Label1', 'Label3', 'Label2', 'Label3', 'Label1']  # Replace with your actual labels

# Normalize embeddings
normalized_embeddings = normalize(enrolled_embeddings)

# Combine embeddings and labels into a DataFrame
df = pd.DataFrame(normalized_embeddings)
df['label'] = enrolled_labels

# Extract the sorted embeddings and labels
sorted_df = df.sort_values(by='label').reset_index(drop=True)
sorted_embeddings = sorted_df.drop(columns='label').values
sorted_labels = sorted_df['label'].values

# Compute cosine similarity matrix for sorted embeddings
cosine_sim_matrix = np.dot(sorted_embeddings, sorted_embeddings.T)

# Prepare unique labels for the axis
unique_labels = []
unique_tickvals = []
for idx, label in enumerate(sorted_labels):
    if label not in unique_labels:
        unique_labels.append(label)
        unique_tickvals.append(idx)

# Plot the heatmap
heatmap = go.Heatmap(z=cosine_sim_matrix, colorscale='Viridis')

# Setup figure with labels on the axes
fig = go.Figure(data=heatmap)

fig.update_layout(
    title="Cosine Similarity Matrix",
    xaxis_title="Image Index",
    yaxis_title="Image Index",
    font=dict(size=14),
    width=1000,
    height=1000,
    margin=dict(t=100, r=100, b=100, l=100),
    xaxis=dict(
        tickmode='array',
        tickvals=unique_tickvals,
        ticktext=unique_labels,
        tickangle=90
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=unique_tickvals,
        ticktext=unique_labels
    ),
)

fig.show()